---
title: Expressions and Arithmetic
abstract: |
    This notebook explains how to read and write basic Python expressions for arithmetic operations. Much like mathematical expressions, the precise meanings of Python expressions must consider the precedence and associativity of the operators. 
---

In [ ]:
import ast  # for AST
import sys
from dis import dis
from ipywidgets import interact, fixed

%load_ext divewidgets
%load_ext jupyter_ai
%ai update chatgpt dive:chat

## Floating Point Numbers

Not all numbers are integers. In Enginneering, we often need to use fractions.

**How to enter fractions in a program?**

In [ ]:
x = -0.1  # decimal number
y = -1.0e-1  # scientific notation
z = -1 / 10  # fraction
x, y, z, type(x), type(y), type(z)

**What is the type `float`?**

- `float` corresponds to the [*floating point* representation](https://en.wikipedia.org/wiki/Floating-point_arithmetic#Floating-point_numbers).  
- A `float` is stored exactly the way we write it in [scientific notation](https://en.wikipedia.org/wiki/Scientific_notation): 

$$
\overbrace{-}^{\text{sign}} \underbrace{1.0}_{\text{mantissa}\kern-1em}e\overbrace{-1}^{\text{exponent}\kern-1em}=-1\times 10^{-1}
$$

::::{seealso}

An efficient implementation is more complicated. Try the [IEEE-754 Floating Point Converter](https://www.h-schmidt.net/FloatConverter/IEEE754.html) for *single-precision* floating point number:

- Starting from the number 0, click the button `+1` to find the smallest positive number.
- Find the largest and smallest representable floating point numbers.

::::

Integers in mathematics may be regarded as a `float` instead of `int`:

In [ ]:
type(1.0), type(1e2)

You can also convert an `int` or a `str` to a `float`.

In [ ]:
float(1), float("1")

**Is it better to store an integer as `float`?**

Python stores a [floating point](https://docs.python.org/3/library/sys.html#sys.float_info) with finite precision, usually *64-bit/double* precision:

In [ ]:
sys.float_info

It cannot accurately represent a number larger than the `max`:

In [ ]:
sys.float_info.max * 2

The precision also affects the check for equality.

In [ ]:
(
    1.0 == 1.0 + sys.float_info.epsilon * 0.5,  # returns true if equal
    1.0 == 1.0 + sys.float_info.epsilon * 0.6,
    sys.float_info.max + 1 == sys.float_info.max,
)

Another issue with float is that it may keep more decimal places than desired.

In [ ]:
1 / 3

**How to [round](https://docs.python.org/3/library/functions.html#round) a floating point number to the desired number of decimal places?**

In [ ]:
round(2.665, 2), round(2.675, 2)

::::{card}
:header: Why 2.675 rounds to 2.67 instead of 2.68?

- A `float` is actually represented in binary.  
- A decimal fraction [may not be represented exactly in binary](https://docs.python.org/3/tutorial/floatingpoint.html#tut-fp-issues).

::::

::::{exercise} binary representations of fractions
:label: ex:float

Use the [IEEE-754 Floating Point Converter](https://www.h-schmidt.net/FloatConverter/IEEE754.html) to find out the value actually stored in float for 2.675.

::::

YOUR ANSWER HERE

The `round` function can also be applied to an integer.

In [ ]:
round(150, -2), round(250, -2)

::::{card}
:header: Why 250 rounds to 200 instead of 300?

- Python 3 implements the default rounding method in [IEEE 754](https://en.wikipedia.org/wiki/IEEE_754#Rounding_rules).

::::

::::{seealso} Arbitrary precision arithmetic
:class: dropdown

Indeed, computers can perform arbitrary precision arithmetic such as the quadratic equation solver in this Maxima notebook:
> [](Maxima.ipynb)

::::

In [ ]:
%%ai chatgpt -f text
Explain in one paragraph how arbitrary precision arithmetic works in a computer despite the memory being finite. 
If we can have arbitrary precision arithmetic, why do we need floats?

## Operators

The followings are common operators you can use to form an expression in Python:

| Operator  |   Operation    | Example |
| --------: | :------------- | :-----: |
| unary `-` | Negation       |  `-y`   |
|       `+` | Addition       | `x + y` |
|       `-` | Subtraction    | `x - y` |
|       `*` | Multiplication |  `x*y`  |
|       `/` | Division       |  `x/y`  |

- `x` and `y` in the examples are called the *left and right operands* respectively.
- The first operator is a *unary operator*, which operates on just one operand.   
    (`+` can also be used as a unary operator, but it is not very useful.)
- All other operators are *binary operators*, which operate on two operands.

Python also supports some more operators:

| Operator |    Operation     | Example |
| -------: | :--------------- | :-----: |
|     `//` | Integer division | `x//y`  |
|      `%` | Modulo           |  `x%y`  |
|     `**` | Exponentiation   | `x**y`  |

The following demonstrates the operations of binary operators:

In [ ]:
binary_operators = {
    "+": " + ",
    "-": " - ",
    "*": "*",
    "/": "/",
    "//": "//",
    "%": "%",
    "**": "**",
}


@interact(operand1=r"10", operator=binary_operators, operand2=r"3")
def binary_operation(operand1, operator, operand2):
    expression = f"{operand1}{operator}{operand2}"
    value = eval(expression)
    print(
        f"""{'Expression:':>11} {expression}\n{'Value:':>11} {value}\n{'Type:':>11} {type(value)}"""
    )

::::{note} What does the modulo operator `%` do?

You can think of it as computing the remainder, but the [truth](https://docs.python.org/3/reference/expressions.html#binary-arithmetic-operations) is more complicated. Try using fractions and negative numbers for the operands.

:::{seealso}
:class: dropdown

The operator implements the [modular arithmetic](https://en.wikipedia.org/wiki/Modular_arithmetic) in Mathematics. The second operand is called the modulus, which means measure in Latin.

:::

::::

:::{exercise} division
:label: ex:division

What is the difference between `/` and `//`?

:::

YOUR ANSWER HERE

::::{exercise} multiplication
:label: ex:multiplication

What does `'abc' * 3` mean? What about `10 * 'a'`?

::::

YOUR ANSWER HERE

::::{exercise} operand types
:label: ex:operand-type

How can you change the default operands (`10` and `3`) for different operators so that the overall expression has type `float`? Do you need to change all the operands to `float`?

::::

YOUR ANSWER HERE

## Operator Precedence and Associativity

An expression can consist of a sequence of operations performed in a row. For instance, does `x + y * z` means

1. `(x + y) * z` or
2. `x + (y * z)`?

In other words, should the operand `y` be associated with `+` or `*`. In Mathematics, multiplication has higher priority, and so `y` should be associated with `*`. This rule also applies to the Python expression. The association can be seen explicitly from the [Abstract Syntax Tree (AST)](https://docs.python.org/3/library/ast.html):

In [ ]:
print(ast.dump(ast.parse("x + y*z", mode='eval'), indent=4))

**How to determine order of operation?**

Like arithmetics, an operand is associated with an operator according to the following list of rules in order, and the operator precedence and associativity in [](#tbl:prec-assoc):
1. *Grouping* by parentheses: Operator in the inner grouping first.
1. Operator *precedence/priority*: Operator of higher precedence first.
1. Operator *associativity*:  
    - Left associativity: Operators are grouped from left to right.
    - Right associativity: Operator are grouped from right to left.

::::{table} Operator precedence and associativity
:label: tbl:prec-assoc

|    Operators     | Associativity |
| :--------------- | :-----------: |
| `**`             |     right     |
| `-` (unary)      |     right     |
| `*`,`/`,`//`,`%` |     left      |
| `+`,`-`          |     left      |
::::

For instance, `x + y - z` is equivalent to `((x + y) - z)` because both `+` and `-` are left associative, and so the operands are grouped with the operators from left to right.

In [ ]:
print(ast.dump(ast.parse("x + y - z", mode='eval'), indent=4))

As another example, `x ** y ** z` is equivalent to `(x ** (y ** z))` because `**` is right associative, and so the operands are grouped with the operators from right to left.

In [ ]:
print(ast.dump(ast.parse("x ** y ** z", mode='eval'), indent=4))

::::{exercise}
:label: ex:precedence1

Explain the value of the expression `-10 ** -2*3`. In particular, why is the second unary operator `-` evaluated first before `**` even though `**` has a higher precedence?

::::

YOUR ANSWER HERE

In [ ]:
%%ai chatgpt -f text
Explain the value of the expression -10 ** -2*3. 
In particular, why is the second unary operator - evaluated first before ** even though ** has a higher precedence?

::::{exercise} 
:label: ex:precedence2

To avoid confusion in the order of operations, we should follow the [style guide](https://www.python.org/dev/peps/pep-0008/#other-recommendations) when writing expression. What is the proper way to write `-10 ** 2*3`? 

:::{tip}

You can use the [code formatter](https://jupyterlab-code-formatter.readthedocs.io/) in JupyterLab to apply the correct programming styles to your code.
:::

::::

In [ ]:
# YOUR CODE HERE
raise NotImplementedError

## Augmented Assignment Operators

For convenience, Python defines the [augmented assignment operators](https://docs.python.org/3/reference/simple_stmts.html#grammar-token-augmented-assignment-stmt) such as `+=`, where  
> `x += 1` means `x = x + 1`.

The following widgets demonstrate other augmented assignment operators.

In [ ]:
@interact(
    initial_value=fixed(r"10"),
    operator=["+=", "-=", "*=", "/=", "//=", "%=", "**="],
    operand=fixed(r"2"),
)
def binary_operation(initial_value, operator, operand):
    assignment = f"x = {initial_value}\nx {operator} {operand}"
    _locals = {}
    exec(assignment, None, _locals)
    print(f"Assignments:\n{assignment:>10}\nx: {_locals['x']} ({type(_locals['x'])})")

::::{note}

Starting from Python 3.8, there is an [assignment expression](https://docs.python.org/3/whatsnew/3.8.html#assignment-expressions) using the operator `:=`. Unlike the (augmented) assignment operators, the operator returns the value assigned.

::::

In [ ]:
y = 3*(x := 15)
x, y

::::{exercise}
:label: ex:augmented-assignment

Can we create an expression using (augmented) assignment operators? Try running the code to see the effect.

::::

In [ ]:
%%optlite -l -h 400
3*(x = 15)

YOUR ANSWER HERE

## String Formatting

**Can we round a `float` or `int` for printing but not calculation?**

This is possible with format strings:

In [ ]:
x = 10000 / 3
print("x ≈ {:.2f} (rounded to 2 decimal places)".format(x))
x

- `{:.2f}` is a [*replacement field*][repf] or place holder
- that gets replaced by a string 
- that represents the argument `x` of `format` 
- according to the [format specification][fspec] `.2f`, i.e.,  
  a decimal floating point number rounded to 2 decimal places.
  
[repf]: https://docs.python.org/3/library/string.html#format-string-syntax
[fspec]: https://docs.python.org/3/library/string.html#format-specification-mini-language

::::{exercise} format `float`
:label: ex:format-float

Play with the following widget to learn the effect of different format specifications. In particular, print `10000/3` as `3,333.33`.

::::

In [ ]:
@interact(
    x="10000/3",
    align={"None": "", "<": "<", ">": ">", "=": "=", "^": "^"},
    sign={"None": "", "+": "+", "-": "-", "SPACE": " "},
    width=(0, 20),
    grouping={"None": "", "_": "_", ",": ","},
    precision=(0, 20),
)
def print_float(x, sign, align, grouping, width=0, precision=2):
    format_spec = (
        f"{{:{align}{sign}{'' if width==0 else width}{grouping}.{precision}f}}"
    )
    print("Format spec:", format_spec)
    print("x ≈", format_spec.format(eval(x)))

In [ ]:
# YOUR CODE HERE
raise NotImplementedError

String formatting is useful for different data types other than `float`.  
E.g., consider the following program that prints a time specified by some variables.

In [ ]:
# Some specified time
hour = 12
minute = 34
second = 56

print("The time is " + str(hour) + ":" + str(minute) + ":" + str(second) + ".")

Imagine you have to show also the date in different formats.  
The code can become very hard to read/write because 
- the message is a concatenation of multiple strings and
- the integer variables need to be converted to strings.

Omitting `+` leads to syntax error. Removing `str` as follows also does not give the desired format.

In [ ]:
print("The time is ", hour, ":", minute, ":", second, ".")  # note the extra spaces

To make the code more readable, we can use the `format` function as follows.

In [ ]:
message = "The time is {}:{}:{}."
print(message.format(hour, minute, second))

::::{note}

- We can have multiple *place-holders* `{}` inside a string.
- We can then provide the contents (any type: numbers, strings..) using the `format` function, which
- substitutes the place-holders by the function arguments from left to right.
::::

According to the [string formatting syntax](https://docs.python.org/3/library/string.html#format-string-syntax), we can change the order of substitution using 
- indices *(0 is the first item)* or 
- names inside the placeholder `{}`:

In [ ]:
print("You should {0} {1} what I say instead of what I {0}.".format("do", "only"))
print("The surname of {first} {last} is {last}.".format(first="John", last="Doe"))

We can also evaluate variables inside the replacement field:

In [ ]:
yyyy, mm = "2024", "09"

In [ ]:
f"""{yyyy}{mm}CS1302
Intro to Comp Progm'g"""

::::{seealso}

The above multiline string may look familiar to you:

![](cs1302.svg)

`f"..."` is called an [f-string](https://docs.python.org/3/tutorial/inputoutput.html#tut-f-strings). It is a syntax specific to python introduced in [PEP 498](https://peps.python.org/pep-0498/).

::::

::::{exercise} f-string
:label: ex:f-string

Play with the following widget to learn more about the formating specification.  
1. What happens when `align` is none but `fill` is `*`?
1. What happens when the `expression` is a multi-line string?

::::

In [ ]:
@interact(
    expression=r"'ABC'",
    fill="*",
    align={"None": "", "<": "<", ">": ">", "=": "=", "^": "^"},
    width=(0, 20),
)
def print_object(expression, fill, align="^", width=10):
    format_spec = f"{{:{fill}{align}{'' if width==0 else width}}}"
    print("Format spec:", format_spec)
    print("Print:", format_spec.format(eval(expression)))

YOUR ANSWER HERE